# Project 2


### This is the first part of code for Project 2.  It web scrapes Glassdoor of companies in Seatte that employ data analystics professionals.  Selenium was used as Glassdoor does not permit BeautifulSoup.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import os
chromedriver = "/Users/ljborton/Downloads/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)

In [ ]:
driver.get("https://www.glassdoor.com/index.htm")

Login with your personal info to get benefit info and click on notifications button.  

In [ ]:
# The code below gives a list of all data analytics jobs in Seattle (if your default city is Seattle.)
query = driver.find_element_by_xpath("//input[@id='sc.keyword']")
query.send_keys("Data Analytics")
query.send_keys(Keys.RETURN)

Glassdoor will only allow you to page through 30 pages (30 jobs/page), then quits.  I grabbed the first 900 
and called it "list_of_nine". 

In [ ]:
'''Loops though all 30 jobs on one page, then clicks to next page and repeats.  Page 30 is not scraped as it 
produces an error when it hits the "next button".  I could/should have fixed this.

list_of_nine = []
for i in range (1,30):
    for j in range(1,31):        
        job_xpath='//*[@id="MainCol"]/div/ul/li[{}]/div[2]/div[2]'.format(j)
        job_xpath = '//*[@id="MainCol"]/div[1]/ul/li[{}]/div[2]/div[2]/div'.format(j)
        job= driver.find_element_by_xpath(job_xpath).text 
        list_of_nine.append(job)
    next_button_xpath='//*[@id="FooterPageNav"]/div/ul/li[7]/a'  
    next_button= driver.find_element_by_xpath(next_button_xpath)
    next_button.click()
time.sleep(2) 

I went back later and manually filtered by industry which gave less than 900 jobs each. I did not automate this 
as Glassdoor's filter did not  seem stable (didn't reset itself at times). I also noted the number of jobs 
listed per filter as it was needed to determine how the "next page" button would be coded.  In 
addition, the progress needs to be watched as popups would appear randomly.  After concatenating and removing 
duplicates, this list produced roughly the same companies; therefore I did not change from my "list_of_nine".

In [ ]:
# industry+num had to be input manually
# industry_pg gives the number of full pages (30 jobs) and industry_rem gives the number of jobs on the last page.
industry_pg=int(industry_num//30)
industry_rem=industry_num%30

''' The code below scrapes all the jobs on the first page, determine how the "next page" button is coded
(if there is one), and continues scraping to the end of the list.  I should have incorporated the industry_rem
for the last page as it now repeats the last job until it reaches 30 jobs.  I knew I would filter out duplicates
though so it didn't really matter.
''' 

list_of_datac = []
for j in range(1,31):     
    job_xpath = '//*[@id="MainCol"]/div[1]/ul/li[{}]/div[2]/div[2]/div'.format(j)
    job = driver.find_element_by_xpath(job_xpath).text 
    list_of_datac.append(job)
    nextbox=0
    if industry_num >30 and industry_num<=60:
        nextbox = 2
        next_button_xpath='//*[@id="FooterPageNav"]/div/ul/li[4]/a'
    elif industry_num > 60 and industry_num<=90:
        nextbox = 3
        next_button_xpath='//*[@id="FooterPageNav"]/div/ul/li[5]/a'
    elif industry_num >90 and industry_num<=120:
        nextbox = 4
        next_button_xpath='//*[@id="FooterPageNav"]/div/ul/li[6]/a'
    elif industry_num >120:
        nextbox = 5
        next_button_xpath='//*[@id="FooterPageNav"]/div/ul/li[7]/a'
    next_button= driver.find_element_by_xpath(next_button_xpath)
    next_button.click() 

In [ ]:
# Save as pickle
import pickle as pkl
with open('list_of_nine.pkl', 'wb') as picklefile:
    pkl.dump(list_of_nine, picklefile)

In [10]:
# Bring in from pickle
import pickle as pkl
with open("list_of_nine.pkl", 'rb') as picklefile: 
    list_of_nine = pkl.load(picklefile)
list_of_nine[0:6]

['Swisslog Healthcare – Kirkland, WA',
 'FLEXE, Inc. – Seattle, WA',
 'Robert Half – Seattle, WA',
 'Amazon – Seattle, WA',
 'Point B – Seattle, WA',
 'Visionary Integration Professionals – Seattle, WA']

In [12]:
# Isolate the company only, remove city
list_of_nine = [ x.split("–")[0] for x in list_of_nine]
print(len(list_of_nine))

870
870


In [13]:
#Many duplicate companies.  Remove them
setnine=set(list_of_nine)
print(len(setnine))

294


The progress below also needs to be watched as the code fails and needs to restarted if it can't find a data object.
I should have put a NoSuchElementException after each find_element_by_xpath...

In [ ]:
'''Grabbing the data for each company in "listnine".  The code gets to the main page, clicks 
on the "jobs/companies/salaries filter" button, selects "company", inputs a company, directs to the company
overview page (checks to make sure it isn't on it already), then gets to the company information page to scrape 
company data.  It tests to see if a star rating exists, and then goes to the company benefits page to grab 
benefit ratings.  It also tests to make sure a benefit rating exists.  Then repeats with the next company.
'''

from selenium.common.exceptions import NoSuchElementException
listnine=list(setnine)
sizel=[]
starl=[]
headql=[]
foundl=[]
industryl=[]
privatel=[]
revenuel=[]
ceol=[]
benefitl=[]
for names in setnine:
    driver.get("https://www.glassdoor.com/index.htm")
    main_button_xpath='//*[@id="SiteSrchTop"]/form/div/ul'
    main_button = driver.find_element_by_xpath(main_button_xpath)
    main_button.click()

    main_company_button_xpath='//*[@id="SiteSrchTop"]/form/div/ul/li[2]/span'
    main_company_button = driver.find_element_by_xpath(main_company_button_xpath)
    main_company_button.click()   
    time.sleep(2)
    
    query_xpath = "//input[@id='sc.keyword']"
    query = driver.find_element_by_xpath(query_xpath)
    query.send_keys(names)
    query.send_keys(Keys.RETURN)
    
    time.sleep(2)
    overview_xpath = '//*[@id="MainCol"]/div[1]/div[2]/div[1]/div[2]/div[1]/a'
       
    try:
        overview = driver.find_element_by_xpath(overview_xpath)
        overview.click()
        time.sleep(1)
        headq_xpath= '//*[@id="EmpBasicInfo"]/div[1]/div/div[2]'
        headq=driver.find_element_by_xpath(headq_xpath).text
        headql.append(headq)
    except NoSuchElementException:
        headq_xpath= '//*[@id="EmpBasicInfo"]/div[1]/div/div[2]'
        headq=driver.find_element_by_xpath(headq_xpath).text
        headql.append(headq)
          
    size_xpath='//*[@id="EmpBasicInfo"]/div[1]/div/div[3]/span'
    size=driver.find_element_by_xpath(size_xpath).text
    sizel.append(size)
     
    found_xpath='//*[@id="EmpBasicInfo"]/div[1]/div/div[4]/span'
    found=driver.find_element_by_xpath(found_xpath).text
    foundl.append(found)

    private_xpath='//*[@id="EmpBasicInfo"]/div[1]/div/div[5]/span'
    private =driver.find_element_by_xpath(private_xpath).text
    privatel.append(private)
                                  
    industry_xpath = '//*[@id="EmpBasicInfo"]/div[1]/div/div[6]/span'         
    industry = driver.find_element_by_xpath(industry_xpath).text
    industryl.append(industry)
    
    revenue_xpath='//*[@id="EmpBasicInfo"]/div[1]/div/div[7]/span'
    revenue = driver.find_element_by_xpath(revenue_xpath).text 
    revenuel.append(revenue)
                                
    ceo_xpath='//*[@id="EmpStats_Approve"]'    
    ceo=driver.find_element_by_xpath(ceo_xpath).text
    ceol.append(ceo)
    
    star_xpath = '//*[@id="MainCol"]/div[1]/div[2]/div[1]/div[2]/div[2]/div/span/span[1]'
    
    try:
        star = driver.find_element_by_xpath(star_xpath).text
        starlist.append(star)
    except NoSuchElementException:
        star= 'NA'
        starlist.append(star)
      
   #click to benefits page to get benefits rating
    ben_button_xpath='//*[@id="EmpLinksWrapper"]/div/a[6]/span[1]'
    ben = driver.find_element_by_xpath(ben_button_xpath)
    ben.click()

    benefit_xpath= '//*[@id="MainCol"]/div[1]/div[1]/div[2]/div[1]'    
    
    try:
        benefit = driver.find_element_by_xpath(benefit_xpath).text
        benefitlist.append(star)
    except NoSuchElementException:
        benefit= 'NA'
        benefitlist.append(benefit)

In [ ]:
'''After error messages, it was found that some company pages had a "part of" field added that shifted
some data into the wrong columns.  These rows were deleted.  They are shown with their index number.

Accenture [40]
Alibaba Group Holding [75]
Murtech Consulting[101]
SM Diversity [109]
Monitor Deloitte listnine[82] 
RSM USLLP listnine [222]
Tyler Technologies [245]
Golock Global [290]
'''

In [ ]:
#This is to get the list of al the companies that didn't have problems
listc1= listnine [0:21]
listc2= listnine [21:40]
listc3= listnine [41:61]
listc4= listnine [61:74]
listc5= listnine [76:82]
listc6= listnine [83:101]
listc102= listnine [102:109]
listc110= listnine [110:222]
listc223= listnine [223:244]
listc246= listnine [246:289]
listc291= listnine [291:]

In [ ]:
# Concatenating the data
ceo900 = (ceo1 +ceo2 + ceo3+ceo4+ceo5+ceo6+ceo102+ceo110+ceo223+ceo246+ceo7)
star900 = (starlist1 + starlist2 + starlist3+starlist4+starlist5+starlist6+starlist102+starlist110+starlist223+starlist246+starlist7)
benefit900 = (benefitlist1 + benefitlist2 + benefitlist3+benefitlist4+benefitlist5+benefitlist6+benefitlist102+benefitlist110+benefitlist223+benefitlist246+benefitlist7)
revenue900 = (revenuelist1 + revenuelist2 +revenuelist3+revenuelist4+revenuelist5+revenuelist6+revenuelist102+revenuelist110+revenuelist223+revenuelist246+revenuelist7)
industry900 = (industrylist1 + industrylist2 +industrylist3+industrylist4+industrylist5+industrylist6+industrylist102+industrylist110+industrylist223+industrylist246+industrylist7)
size900 = (sizelist1 + sizelist2 +sizelist3+sizelist4+sizelist5+sizelist6+sizelist102+sizelist110+sizelist223+sizelist246+sizelist7)
private900 = (privatelist1 + privatelist2 +privatelist3+privatelist4+privatelist5+privatelist6+privatelist102+privatelist110+privatelist223+privatelist246+privatelist7)
headq900 = (headqlist1 + headqlist2 +headqlist3+headqlist4+headqlist5+headqlist6+headqlist102+headqlist110+headqlist223+headqlist246+headqlist7)
found900 = (foundlist1 +foundlist4+foundlist5+foundlist6+foundlist102+foundlist110+foundlist223+foundlist246+foundlist7)
company900 = (listc1+ listc2+ listc3+ listc4+listc5+ listc6+ listc102+listc110+ listc223 + listc246 + listc291)

In [ ]:
import pandas as pd
df900 = pd.DataFrame(
    {'Company': company900, 'Reviews': star900, 'CEO_Reviews': ceo900,'Founded': found900, 'Headquarters':headq900, 
     'Private_Public' : private900, 'Benefit_Rating':benefit900, 'Revenue': revenue900, 
     'Industry': industry900, 'Size': size900})

In [ ]:
# Save as pickle
import pickle as pkl
with open('df900.pkl', 'wb') as picklefile:
    pkl.dump(list_of_nine, picklefile)

In [ ]:
driver.close()